Es ist möglich, fehlende Python-Pakete in Google Colab nachzuinstallieren. </br>
Das macht man in Python üblicherweise über den Paketmanager pip.</br> 
Man kann mit Präfix "!" Shell-Kommandos ausführen und so Pakete installieren.

In [1]:
!pip install tabula-py

     |████████████████████████████████| 11.7MB 4.2MB/s 


Aus der Dokumentation (https://tabula-py.readthedocs.io/en/latest/): </br> tabula (ein Programm zum Extrahieren von Tabellen aus PDFs) ist ein Python-Wrapper für eine Java-Bibliothek. Das heißt, wir benutzen Python Syntax, aber die eigentliche Arbeit wird von Java Programmen gemacht. Das findet man öfter. In diesem Fall ist es gut zu prüfen, ob die benötigte Java-Version installiert ist.

In [ ]:
!java -version

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


Nachdem das Python-Modul installiert wurde, können wir es in unserem Code importieren. </br> Erst danach können wir seine Objekte und Funktionen verwenden.

In [6]:
import tabula

tabula.environment_info()

Python version:
    3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
Java version:
    openjdk version "11.0.9.1" 2020-11-04
OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
tabula-py version: 2.2.0
platform: Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
uname:
    uname_result(system='Linux', node='414badca5934', release='4.19.112+', version='#1 SMP Thu Jul 23 08:00:38 PDT 2020', machine='x86_64', processor='x86_64')
linux_distribution: ('Ubuntu', '18.04', 'bionic')
mac_ver: ('', ('', '', ''), '')
    


Kurze Einführung zu Python Dictionaries: ein Dictionary ist eine Liste von Key/Value Paaren. </br> Man ruft den Value ab, indem man den Key als Index zum Dictionary benutzt.

In [14]:
myPreferences = { "Obst": "Beeren",
                  "Getränk": "Direktsaft", 
                  "Band": "Oscar Peterson Trio"}
print(myPreferences["Getränk"])

Direktsaft


Die Bibliothek tabula verwendet ein anderes Python Modul zur Datenmanipulation, Pandas (siehe z. B. hier: https://pandas.pydata.org/docs/getting_started/intro_tutorials/01_table_oriented.html). Pandas ist in Google Colab standardmäßig installiert. Das Ergebnis der read_pdf Funktion ist ein Dataframe, der funktioniert so ähnlich wie ein Dictionary mit Listen als Values.  

In [11]:
# Beispiel von der oben angegebenen Webseite
import pandas as pd
df = pd.DataFrame({
        "Name": ["Braund, Mr. Owen Harris",
                 "Allen, Mr. William Henry",
                 "Bonnell, Miss. Elizabeth"],
        "Age": [22, 35, 58],
        "Sex": ["male", "male", "female"]}
)
df

,Name,Age,Sex
0,"Braund, Mr. Owen Harris",22,male
1,"Allen, Mr. William Henry",35,male
2,"Bonnell, Miss. Elizabeth",58,female


In [13]:
df["Age"]

0    22
1    35
2    58
Name: Age, dtype: int64

Jetzt lesen wir mal eine Tabelle aus einem PDF ein.

In [18]:
pdf_path = "https://www.rheingau-taunus.de/fileadmin/forms/corona-virus/coronavirus_inzidenz_12-14.pdf"

dfs = tabula.read_pdf(pdf_path, stream=True, pages=1)

# dfs ist eine Liste von Dataframes, also von Tabellen.
# Man erkennt die Liste an den [] Klammern außen.
dfs

[      Stadt | Gemeinde Neufälle der letzten 7 Tage
 0            Aarbergen                          24
 1       Bad Schwalbach                          17
 2             Eltville                          39
 3           Geisenheim                          21
 4            Heidenrod                           7
 5           Hohenstein                           5
 6           Hünstetten                           6
 7              Idstein                          36
 8             Kiedrich                          17
 9                Lorch                           1
 10       Niedernhausen                          11
 11     Oestrich-Winkel                          62
 12  Rüdesheim am Rhein                          19
 13        Schlangenbad                           0
 14         Taunusstein                          85
 15             Waldems                           5
 16              Walluf                          31
 17        Kreis gesamt                         386
 18      Kre

In [20]:
# Wir wissen natürlich, dass die Liste nur einen Eintrag hat.
t1 = dfs[0]
t1

,Stadt | Gemeinde,Neufälle der letzten 7 Tage
0,Aarbergen,24
1,Bad Schwalbach,17
2,Eltville,39
3,Geisenheim,21
4,Heidenrod,7
5,Hohenstein,5
6,Hünstetten,6
7,Idstein,36
8,Kiedrich,17
9,Lorch,1


Pandas hat Funktionen zum Selektieren und Joinen, ähnlich wie wir sie in SQL auch benutzen.

In [21]:
t1[t1['Stadt | Gemeinde'].isin(['Kreis gesamt', 'Kreis Inzidenz'])]

,Stadt | Gemeinde,Neufälle der letzten 7 Tage
17,Kreis gesamt,386
18,Kreis Inzidenz,"206,2"


In [22]:
t1[t1['Stadt | Gemeinde']=='Kreis gesamt']

,Stadt | Gemeinde,Neufälle der letzten 7 Tage
17,Kreis gesamt,386


Mit der funktion .iloc()[Zeile] können wir statt über einen Key auch numerisch selektieren.

In [24]:
t1[t1['Stadt | Gemeinde']=='Kreis gesamt'].iloc()[0]['Neufälle der letzten 7 Tage']

'386'

Zum Schluss verpacke ich das noch in eine Python-Funktion:

In [ ]:
def getValue(DF, CompCol, RowVal, ResultCol):
  return DF[DF.iloc[:,CompCol]==RowVal].iloc[0, ResultCol]
  
print(getValue(t1, 0, 'Kreis gesamt', 1))

155
